#Import Libraries

In [ ]:
import pandas as pd
import time
%matplotlib inline
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
import codecs
import ast

import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
username = ''
api_key = ''
chart_studio.tools.set_credentials_file(username=username,api_key=api_key)


import os
import warnings
warnings.simplefilter(action = 'ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
def ignore_warn(*args, **kwargs):
    pass

warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
    """
    Imports libraries & sets source path to read HDF5 files.
    """

import numpy as nm
import sys
import pg
sys.path.append('/content/drive/MyDrive/PROJECT CODE/MSongsDB-master/PythonSrc')

In [ ]:
    """
    Imports libraries for source code.
    """

from MBrainzDB import query as QUERYMB
import hdf5_utils as HDF5
import hdf5_getters as g

#Read HDF5 files and save as local CSV

In [ ]:
import glob
files = glob.glob('/content/drive/MyDrive/PROJECT CODE/MillionSongSubset/*/*/*/*.h5')

In [ ]:
len(files)

10000

In [ ]:
% time
artist_names = []
song_id = []
song_titles =[]
song_durations =[]
song_release_years =[] #year when this song was released, according to musicbrainz.org
song_hotness  =[]
song_tempo = []
song_energies = []
song_danceability = []
song_bars =[]
song_beats = []
song_time_signatures =[]
song_tatum =[]
song_modes=[]
song_keys=[]
artist_familiarty = []
artist_hotness = []

for f in files:
    
    h5 = HDF5.open_h5_file_read(f)
    
    song_ids = g.get_song_id(h5)
    song_title = g.get_title(h5)
    song_duration = g.get_duration(h5)
    song_release_year = g.get_year(h5)
    song_hot = g.get_song_hotttnesss (h5)
    song_speed = g.get_tempo(h5)
    song_energy = g.get_energy(h5)
    artist_name = g.get_artist_name(h5)
    song_dance = g.get_danceability(h5)
    song_bar = g.get_bars_start(h5)
    song_beat = g.get_beats_start(h5)
    song_time_signature = g.get_time_signature(h5)
    song_tat = g.get_tatums_start(h5)
    song_mode = g.get_mode(h5)
    song_key = g.get_key(h5)
    artist_familar = g.get_artist_familiarity(h5)
    artist_hot = g.get_artist_hotttnesss(h5)
    
    song_id.append(song_ids)
    song_titles.append(song_title)
    song_durations.append(song_duration)
    song_release_years.append(song_release_year)
    song_hotness.append(song_hot)
    song_energies.append(song_energy)
    song_tempo.append(song_speed)
    artist_names.append(artist_name)
    song_danceability.append(song_dance)
    song_bars.append(song_bar)
    song_beats.append(song_beat)
    song_time_signatures.append(song_time_signature)
    song_tatum.append(song_tat)
    song_modes.append(song_mode)
    song_keys.append(song_key)
    artist_familiarty.append(artist_familar)
    artist_hotness.append(artist_hot)
    
    h5.close()

In [ ]:
data = {'song_ids':song_ids
        ,'song_titles':song_titles
        ,'song_durations':song_durations
        ,'song_realease_years':song_release_years
        ,'artist_names':artist_names
        ,'song_hotness':song_hotness
        ,'song_tempo':song_tempo
        ,'song_energies':song_energies
        ,'song_danceability':song_danceability
        ,'song_bars':song_bars
        ,'song_beats':song_beats
        ,'song_time_signatures':song_time_signatures
        ,'song_tatum':song_tatum
        ,'song_modes':song_modes
        ,'song_keys':song_keys
        ,'artist_familiarty':artist_familiarty
        ,'artist_hotness':artist_hotness
       }

In [ ]:
df=pd.DataFrame(data)
df.to_csv('/content/drive/MyDrive/PROJECT CODE/music.csv', sep=',')

#**Create Song Popularity Dataset**
**Missing data**
> song_hotness            4,352

Length of Data before dropping null values: 10,000
<br /> Length of Data after dropping null values: 5,648

*Song Popularity* variable is created with "Yes" being a value greater than the mean of song hotness greater & "No" being a value less than or equal to the mean of song hotness.

*tatum_number*, *beats_number*, *bars_number* are all the amount of numbers in their respective columns for the main dataframe.

### Combine track data to song data

In [ ]:
df = pd.read_table('/content/drive/MyDrive/PROJECT CODE/music.csv', sep=',')
df['song_ids'] = df['song_ids'].apply(ast.literal_eval).str.decode("utf-8")
df['song_titles'] = df['song_titles'].apply(ast.literal_eval).str.decode("utf-8")
df['artist_names'] = df['artist_names'].apply(ast.literal_eval).str.decode("utf-8")
df.head()

In [ ]:
track = pd.read_table('/content/drive/MyDrive/PROJECT CODE/data/unique_tracks.txt', sep='<SEP>', header=None)
track.columns = ['track_id', 'song_ids', 'artist_name', 'title']
track = track.drop(columns=['artist_name','title'])

songs = pd.merge(df, track, how='left', on='song_ids')

In [ ]:
df1 = songs[songs['song_realease_years']!=0]
df1 = df1[df1.song_realease_years > 1965]
df1 = df1[df1['song_hotness']!=0]
df1 = df1[df1['song_hotness'].notna()]
df1.rename(columns = {'artist_familiarty':'artist_familiarity'}, inplace = True)
df1.isna().sum()

In [ ]:
df1['beats_number'] = [len(x) for x in df1.song_beats]
df1['bars_number'] = [len(x) for x in df1.song_bars]
df1['tatum_number'] = [len(x) for x in df1.song_tatum]
df1 = df1.drop(columns=['Unnamed: 0'])

song_hotness_mean = df1.song_hotness.mean() #0.5026434055509752

conditions = [
    (df1['song_hotness'] < 0.5026),
    (df1['song_hotness'] >= 0.5026)
]

values = ['No', 'Yes']

df1['isPopular'] = np.select(conditions, values)
df1.head()

# **Create User Recommendation Dataset**
**Missing data**
> 
title                 42
<br />genre            9,809,858
<br />year             4,892,722
<br />artist_id       16,472,410
<br />longitude       16,472,410
<br />latitude        16,472,410
<br />location        16,472,719

Length of Data before dropping null values: 24,982,827
<br />Length of Data after dropping null values & subgenre: 4,632,548


In [58]:
users = pd.read_csv('/content/drive/MyDrive/PROJECT CODE/data/user_data.txt', sep='\t', header=None)
users.columns = ['user_id', 'song_ids', 'listen_count']
users = users.sample(frac=0.50, replace=True, random_state=1)


track = pd.read_table('/content/drive/MyDrive/PROJECT CODE/data/unique_tracks.txt', sep='<SEP>', header=None)
track.columns = ['track_id', 'song_ids', 'artist_name', 'title']


genrecols = ['track_id', 'genre', "subgenre"]
genre = pd.read_table('/content/drive/MyDrive/PROJECT CODE/data/track_genre.cls', sep='\t', header=None, skiprows=7, names = genrecols)

year = pd.read_csv('/content/drive/MyDrive/PROJECT CODE/data/tracks_per_year.txt', sep='<SEP>', header=None)
year.columns = ['year', 'track_id', 'artist_name', 'title']
year = year.drop(columns=['artist_name','title'])

music_df = pd.merge(users, track, how='left', on='song_ids')
music_df = pd.merge(music_df, genre, how='left', on='track_id')
music_df = pd.merge(music_df, year, how='left', on='track_id')

In [59]:
artist = pd.read_table("/content/drive/MyDrive/PROJECT CODE/data/artist_location.txt", sep='<SEP>', header=None)
artist.columns = ['artist_id','longitude','latitude', 'artist_name','location']
song_df = pd.merge(music_df, artist, how='left', on='artist_name')

In [ ]:
song_df = song_df.drop(columns=['subgenre'])
song_df.isna().sum()
song_df = song_df.dropna()

#EDA for Song Popularity
Explore data prior to building regression model.

In [ ]:
import plotly.express as px
df2 = df1.groupby(by=["song_realease_years"]).size().reset_index(name="counts")
fig = px.bar(df2, x='song_realease_years', y='counts', color_discrete_sequence=['#7653A3'],
            labels={col:col.replace('_', ' ') for col in df.columns},
             height=600)
fig.update_layout(
    font_family="Droid Serif",
    yaxis_title="Song Count",
    title_text='<b>Songs per Year<b>',
    title_x=0.5,
    titlefont={'size': 24})
        
py.plot(fig,filename='song years countplot',auto_open=False)
fig.show()

In [ ]:
df2 = df1.groupby(by=["isPopular","song_realease_years"]).size().reset_index(name="counts")
fig = px.bar(df2, x="isPopular", y="counts", color_discrete_sequence=['#7653A3'],
  animation_frame="song_realease_years", animation_group="counts", range_y=[0,120])
fig.update_layout(
    font_family="Droid Serif",
    title_text='<b>Popular Songs by Year<b>',
    title_x=0.5,
    titlefont={'size': 24})
py.plot(fig,filename='Popularity by Year',auto_open=False)

fig.show()

In [ ]:
fig = px.scatter_matrix(df1, color = 'song_realease_years',
    dimensions=['song_hotness', 'song_tempo', 'song_time_signatures', 'song_durations', 'artist_hotness'],
    labels={col:col.replace('_', ' ') for col in df.columns},
    width = 1200,
    height = 1000,
    color_continuous_scale= 'purp')
fig.update_layout(
    font_family="Droid Serif",
    title_text='<b>Songs Scatterplot Matrix<b>',
    title_x=0.5,
    titlefont={'size': 24})
fig.update_traces(diagonal_visible=False)
py.plot(fig,filename='song matrix',auto_open=False)
fig.show()

In [ ]:
fig = px.scatter(df1, x="artist_familiarity", y="artist_hotness", color="isPopular",
                 size='song_hotness', hover_data=['isPopular'],
                 labels={col:col.replace('_', ' ') for col in df.columns},
                 color_discrete_sequence=['#7653A3', '#BFA1E6'])
fig.update_layout(title_text='<b>Artist Familiarty & Hotness <b>',
                  title_x=0.5,
                  titlefont={'size': 24},
                  width=700, height=500,
                  xaxis_showgrid=False,
                  yaxis_showgrid=False,                   
                  paper_bgcolor=None,
                  )
py.plot(fig,filename='artist scatterplot',auto_open=False)
fig.show()

In [ ]:
import plotly.figure_factory as ff
df3 = df1[['beats_number', 'bars_number', 'tatum_number', 'song_modes', 'artist_hotness','song_hotness']]
df3.columns = df3.columns.str.replace('_',' ')
corr = df3.corr()
fig = ff.create_annotated_heatmap(
    z=corr.to_numpy().round(2),
    x=list(corr.index.values),
    y=list(corr.columns.values),       
    xgap=3, ygap=3,
    zmin=-1, zmax=1,
    colorscale='purp',
    colorbar_thickness=30,
    colorbar_ticklen=3,
)
fig.update_layout(title_text='<b>Correlation Matrix<b>',
                  title_x=0.5,
                  titlefont={'size': 24},
                  width=550, height=550,
                  xaxis_showgrid=False,
                  xaxis={'side': 'bottom'},
                  yaxis_showgrid=False,
                  yaxis_autorange='reversed',                   
                  paper_bgcolor=None,
                  font_family="Droid Serif")
py.plot(fig,filename='Song Correlation Matrix',auto_open=False)
fig.show()

#EDA for User Recommendation
Explore data prior to creating recommendation engine.

In [ ]:
df3 = song_df[song_df.year > 1965]
df3 = df3.groupby(by=["year"]).size().reset_index(name="counts")
fig = px.bar(df3, x='year', y='counts', color_discrete_sequence=['#7653A3'],
            labels={col:col.replace('_', ' ') for col in song_df.columns},
             height=600)
fig.update_layout(
    font_family="Droid Serif",
    yaxis_title="User Count")
py.plot(fig,filename='USER years countplot',auto_open=False)
fig.show()

In [ ]:
import plotly.express as px
popular_songs = song_df.groupby(['title'])['listen_count'].sum().reset_index(name="listen_count")
popular_songs = popular_songs.sort_values('listen_count', ascending=False).head(20).sort_values('listen_count')
fig = px.bar(popular_songs, x='listen_count', y='title', color_discrete_sequence=['#7653A3'], orientation='h',
             height=600, labels={col:col.replace('_', ' ') for col in song_df.columns},)
fig.update_layout(
    font_family="Droid Serif",
    xaxis_title="Listen Count")
py.plot(fig,filename='Top 20 Most Popular Songs',auto_open=False)
fig.show()

In [ ]:
import plotly.express as px
popular_songs = song_df.groupby(['genre'])['listen_count'].sum().reset_index(name="listen_count")
popular_songs = popular_songs.sort_values('listen_count', ascending=False).head(10).sort_values('listen_count')
fig = px.pie(popular_songs, values='listen_count', names='genre',color_discrete_sequence=px.colors.sequential.Purp_r)
fig.update_layout(
    font_family="Droid Serif",
    xaxis_title="Listen Count")
py.plot(fig,filename='Top 10 Most Popular Genres',auto_open=False)
fig.show()

In [ ]:
import plotly.express as px
popular_songs = song_df.groupby(['artist_name'])['listen_count'].sum().reset_index(name="listen_count")
popular_songs = popular_songs.sort_values('listen_count', ascending=False).head(10).sort_values('listen_count')
fig = px.bar(popular_songs, x='listen_count', y='artist_name', color_discrete_sequence=['#7653A3'], orientation='h',
             height=600, labels={col:col.replace('_', ' ') for col in song_df.columns},)
fig.update_layout(
    font_family="Droid Serif",
    xaxis_title="Listen Count")
py.plot(fig,filename='Top 10 Most Popular Artist',auto_open=False)
fig.show()

#**Song Popularity Regression Model**
Feature importance & reduction through Random Forest Regressor & then predict data with XGBRegressor.

In [ ]:
    """
    Graphs features for dashboard visualization.
    """
X = df1.drop(['song_hotness','isPopular'],axis=1)
X = X.select_dtypes(exclude=[object])
y = df1['song_hotness'].values

model = RandomForestRegressor(n_estimators=100, max_features=5, bootstrap = True)
model.fit(X,y) 

features = X.columns
feature_importances = model.feature_importances_

features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort_values('Importance Score', inplace=True, ascending=False)

fig = px.bar(features_df, x='Features', y='Importance Score', color_discrete_sequence=['#7653A3'],
            labels={col:col.replace('_', ' ') for col in df.columns},
             height=600, text_auto='.2f')
fig.update_layout(
    font_family="Droid Serif")
py.plot(fig,filename='Feature Importance',auto_open=False)
fig.show()

In [ ]:
    """
    Selects 5 features for regression model to increase model perfromance.
    """
def select_features(X_train, y_train, X_test):
	# configure to select a subset of features
	fs = SelectFromModel(RandomForestRegressor(n_estimators=2000, bootstrap = True), max_features=5)
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

In [ ]:
    """
    Runs regression model to predict song hotness.
    """

# define the dataset
X = df1.drop(['song_hotness','isPopular'],axis=1)
X = X.select_dtypes(exclude=[object])
y = df1['song_hotness'].values
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# fit the model
model = XGBRegressor(eta=0.1, max_depth=8,n_estimators= 200)
model.fit(X_train_fs, y_train)
# evaluate the model
y_pred = model.predict(X_test_fs)

[21:55:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
    """
    Gauge model accuracy based on RMSE & MAPE.
    """

rmse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_error(y_test, y_pred)*100


print("RMSE: %f" % (rmse))
#The smaller the RMSE, the better a model is able to fit the data.

print("MAPE: %f" % (mape))
#In other words, the average difference between the predicted value and the actual value is 11.628128%


RMSE: 0.020375
MAPE: 11.628128


In [46]:
    """
    Adds predicted values to main dataframe then saves locally for visualizations.
    """

predicted_data = pd.DataFrame(y_pred, columns=['Predicted_Hotness'])
predicted_data = pd.merge(df1, predicted_data, left_index=True, right_index=True)
predicted_data.to_csv('/content/drive/MyDrive/Personal /predicted_data.csv', header=True, index=False, encoding='utf-8')

#**User Recommendation Model**
Content-Based Filtering <br  />
*predicts what a user like based on what that user like in the past*

In [ ]:
    """
    Label encodes genre.
    """

le = preprocessing.LabelEncoder()
song_df['genre_id'] = le.fit_transform(song_df['genre'])

    """
    Split train & test data.
    """

train_data, test_data = train_test_split(song_df, test_size = 0.3, random_state=1)
number_cols = ['genre_id', 'longitude', 'latitude', 'year']
song_data = train_data.groupby('song_ids')['song_ids','title','artist_name', 'genre', 'genre_id', 'longitude', 'latitude', 'location', 'year'].head(1)

    """
    Standardizes data.
    """

scaler = StandardScaler()
scaler.fit(song_data[number_cols])


In [ ]:
def get_song_list(user_id):
    
    """
    Gets the information of previous songs that a user has listened to.
    """
        
    song_id_list = song_df[song_df['user_id'] == user_id]['song_ids'].tolist()
    return song_data[song_data['song_ids'].isin(song_id_list)]

def get_mean_vector(user_id):
  
    """
    Gets the mean vector for a list of songs.
    """
    
    song_vectors = song_df[song_df['user_id'] == user_id][number_cols] 
    song_vectors['genre_id'] = song_df[song_df['user_id'] == user_id]['genre_id'].mode()[0] # for genre, recommend the mode
    song_matrix = np.array(song_vectors)
    return np.mean(song_matrix, axis=0)

def recommend_songs(user_id, n_songs=10):
  
    """
    Recommends songs based on a list of previous songs that a user has listened to.
    """
    song_history = song_df[song_df['user_id'] == user_id]['song_ids']
    song_list = song_history.tolist()
    
    song_center = get_mean_vector(user_id)
    scaled_data = scaler.transform(song_data[~song_data.song_ids.isin(song_list)][number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])
    
    rec_songs = song_data[~song_data.song_ids.isin(song_list)].iloc[index]
    rec_songs['distance'] = distances[0][index]
    rec_songs['rank'] = rec_songs['distance'].rank(ascending=True, method='first')
    return rec_songs

In [ ]:
get_song_list('ce47ddf4975ed5287adf9591541ca31a61dddb6d')

,song_ids,title,artist_name,genre,genre_id,longitude,latitude,location,year
24972482,SOXGXKY12A8C13A405,Remember When (Side A),The Black Keys,Rock,13,41.08419,-81.51406,"Akron, OH",2008.0
20391366,SOYGKHG12AB01804C8,Crazier,Taylor Swift,Country,1,34.05349,-118.24532,California - LA,2009.0
13841411,SOIFDQD12AB01822F5,Busted,The Black Keys,Rock,13,41.08419,-81.51406,"Akron, OH",2002.0
13462202,SOZCSPA12AB0185672,Aeroplane Blues,The Black Keys,Rock,13,41.08419,-81.51406,"Akron, OH",2004.0
15963344,SOHMSPV12AB0185643,Girl Is On My Mind,The Black Keys,Rock,13,41.08419,-81.51406,"Akron, OH",2004.0
20077426,SOLLDVS12AB0183835,I'll Be Your Man,The Black Keys,Rock,13,41.08419,-81.51406,"Akron, OH",2002.0
21475503,SOYEVUY12A8C145F58,Strange Desire,The Black Keys,Rock,13,41.08419,-81.51406,"Akron, OH",2006.0
4695144,SOJDPNR12A8C13C7AE,Things Ain't Like They Used To Be,The Black Keys,Rock,13,41.08419,-81.51406,"Akron, OH",2008.0
8765408,SOQRMFX12AB0187662,The Moan,The Black Keys,Rock,13,41.08419,-81.51406,"Akron, OH",2003.0
3288897,SOQTPLM12B0B809575,Strange Times,The Black Keys,Rock,13,41.08419,-81.51406,"Akron, OH",2008.0


In [ ]:
%%time
recommend_songs('ce47ddf4975ed5287adf9591541ca31a61dddb6d')[['title','artist_name','genre','distance','rank']]

CPU times: user 3.54 s, sys: 27.4 ms, total: 3.56 s
Wall time: 3.56 s


,title,artist_name,genre,distance,rank
23940285,Long Walk Home,Billy Sheehan,Rock,0.00099,1.0
19452385,Hope,Billy Sheehan,Rock,0.00099,2.0
9606286,The Suspense Is Killing Me,Billy Sheehan,Rock,0.00099,3.0
8770223,Toss It On The Flame,Billy Sheehan,Rock,0.00099,4.0
15034993,Dig A Hole,Billy Sheehan,Rock,0.00099,5.0
20275374,Taj,Billy Sheehan,Rock,0.00099,6.0
14277481,From The Backseat,Billy Sheehan,Rock,0.00099,7.0
14142697,The Lift,Billy Sheehan,Rock,0.00099,8.0
10792447,Back In The Day,Billy Sheehan,Rock,0.00099,9.0
15659394,Don't Look Down,Billy Sheehan,Rock,0.00099,10.0
